In [258]:
import pandas as pd
import numpy as np

### Merge Harvest data to the bridge project Data

In [259]:
db_df = pd.read_csv("https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/final_csv/Final_with_gov_ID.csv")
harvest_df = pd.read_csv("https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/share_of_harvest_sold.csv")

In [260]:
print(db_df.shape)
print(harvest_df.shape)

(4483, 20)
(30, 3)


In [261]:
db_df.drop(['Unnamed: 0','Cell','Form: Form Name','Cell_ID'],axis=1,inplace=True)
harvest_df.drop('Unnamed: 0', axis=1,inplace=True)

In [262]:
# Merge database file on share of harvest in each RW district
df = pd.merge(db_df,harvest_df,on='District',how='inner')

In [263]:
# db_df['District'].nunique()
# df['District'].nunique()
# harvest_df['Mean share of harvest sold'].nunique()
# df['Mean share of harvest sold'].nunique()

### Cleaning and merging of Schools and Gov ID data

In [264]:
schools_df = pd.read_csv("https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/Rwanda_Schools%20(1).csv")

In [265]:
govt_df = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/unique_gov_id_PDS.csv')

In [266]:
govt_df['Sector'] = govt_df['Sector'].str.title()
schools_df['Sector'] = schools_df['Sector'].str.title()
schools_df['District'] = schools_df['District'].str.title()

In [267]:
# Correct spellings in schools data to match govt_data 
govt_to_sch = {
    'Kabagali' : 'Kabagari',
    'Muringa' : 'Mulinga' ,
    'Rugarika' : 'Rugalika',
    'Gishali' : 'Gishari' ,
    'Rugengabari' : 'Rugengabali',
    'Nyagihanga':'Nyagahanga',
    'Niboye': 'Niboyi',
    'Save':'Gatoki',
    'Tumba':'Ntuba'
    }

for key,value in govt_to_sch.items():

  schools_df.loc[schools_df['Sector'] == value,'Sector'] = key

In [268]:
b = [x for x in set(schools_df['Sector']) if x not in set(govt_df['Sector'])]
b

[]

In [269]:
# Change Nyanza-Kibirizi to Nyanza-Kibilizi to match database_df
indy = schools_df[(schools_df['District'] == "Nyanza") & (schools_df['Sector'] == 'Kibirizi')].index

for i in indy:
  schools_df.loc[i,['Sector']] = "Kibilizi"

In [270]:
schools_df.drop('Unnamed: 0',axis=1,inplace=True)

In [271]:
# Correct Sector names in schools data 
il = [294]

for i in il:
  schools_df.loc[i,['Sector']] = 'Bigogwe'

jl = [185]

for j in jl:
  schools_df.loc[j,['Sector']] = 'Ngarama'

In [272]:
# Merge schools data on govt data 
mg = pd.merge(schools_df,govt_df,on=['District','Sector'])

In [273]:
y = [x for x in set(mg['School Name']) if x not in set(schools_df['School Name'])]
y

[]

In [274]:
q = [x for x in set(schools_df['School Name']) if x not in set(mg['School Name'])]
q

[]

In [275]:
schools_df[schools_df['School Name'] == 'College Baptiste De Ngarama']

,School Name,Province,District,Sector
185,College Baptiste De Ngarama,East,Gatsibo,Ngarama


In [276]:
mg[mg['School Name'] == 'EAV BIGOGWE']

,School Name,Province_x,District,Sector,Prov_ID,Province_y,Dist_ID,Sect_ID
411,EAV BIGOGWE,West,Nyabihu,Bigogwe,3,Iburengerazuba,34,3401


In [277]:
mg[mg['School Name'] == 'College Baptiste De Ngarama']

,School Name,Province_x,District,Sector,Prov_ID,Province_y,Dist_ID,Sect_ID
1212,College Baptiste De Ngarama,East,Gatsibo,Ngarama,5,Iburasirazuba,53,5310


In [278]:
# schools_df.groupby(['District','Sector'],as_index=False).count()

In [279]:
# Get the count of schools in each sector 
count_sch = mg.groupby(['Sect_ID']).count()['School Name']
count_sch.head(3)

Sect_ID
1101    2
1102    2
1103    3
Name: School Name, dtype: int64

In [280]:
# merge database df on school count 
test_df = pd.merge(df,count_sch,on='Sect_ID')

In [281]:
test_df['School_count'] = test_df['School Name']

In [282]:
test_df.drop('School Name',axis=1,inplace=True)

In [283]:
current_df = test_df

In [284]:
# Current final df with farm harvest by District 
# and number of schools in each Sector
current_df.shape

(4483, 18)

### Clean and Merge Sector population with the bridge project data

In [285]:
population = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/Rwanda%20Administrative%20Levels%20and%20Codes%20Sector%20Population.csv')

In [286]:
population

,Prov_ID,Province,Dist_ID,District,Sect_ID,Sector,Population of Sector,Unnamed: 7,Unnamed: 8
0,1,Umujyi wa Kigali,11,Nyarugenge,1101,Gitega,"28,728.00",NaN,NaN
1,1,Umujyi wa Kigali,11,Nyarugenge,1102,Kanyinya,"21,859.00",NaN,NaN
2,1,Umujyi wa Kigali,11,Nyarugenge,1103,Kigali,"30,023.00",NaN,NaN
3,1,Umujyi wa Kigali,11,Nyarugenge,1104,Kimisagara,"46,753.00",NaN,NaN
4,1,Umujyi wa Kigali,11,Nyarugenge,1105,Mageregere,"23,407.00",NaN,NaN
...,...,...,...,...,...,...,...,...,...
413,5,Iburasirazuba,57,Bugesera,5711,Nyarugenge,"20,753.00",NaN,NaN
414,5,Iburasirazuba,57,Bugesera,5712,Rilima,"26,803.00",NaN,NaN
415,5,Iburasirazuba,57,Bugesera,5713,Ruhuha,"22,994.00",NaN,NaN
416,5,Iburasirazuba,57,Bugesera,5714,Rweru,"28,782.00",NaN,NaN


In [287]:
current_df.head()

,Province,District,Sector,Bridge Site Name,Project Stage,Project Code,Bridge Type,GPS (Latitude),GPS (Longitude),Individuals Directly Served,Assessment Date,Original_Community_col,Community_Served,Prov_ID,Dist_ID,Sect_ID,Mean share of harvest sold,School_count
0,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 1,Buzi,3,36,3605,20.1,6
1,Western Province,Rusizi,Giheke,Kamigisha,Rejected,1014106,Suspended,-2.42486,28.957259999999998,Unknown,2018-10-29,Community Served 1,Kabuga,3,36,3605,20.1,6
2,Western Province,Rusizi,Giheke,Gacyamo,Rejected,1014108,Suspension,-2.41855,28.969459999999998,Unknown,2018-10-29,Community Served 1,Gacyamo,3,36,3605,20.1,6
3,Western Province,Rusizi,Giheke,Nyakabanda,Rejected,1014105,Suspended,-2.42715,28.95369,Unknown,2018-10-29,Community Served 1,Kanoga,3,36,3605,20.1,6
4,Western Province,Rusizi,Giheke,Impala,Rejected,1014166,Unknown,-2.44246,28.9333,Unknown,2018-10-29,Community Served 1,Impala,3,36,3605,20.1,6


In [288]:
[i for i in set(population['Sect_ID']) if i not in set(govt_df['Sect_ID']) ]

[]

In [289]:
# there are some duplicate values that we can drop. 
# but Sect code 2411 is showing for two different Sectors. We cant drop that one.
# we need to change the sector code for the sector Rwaniro to 2412.
population['Sect_ID'].value_counts()

1108    2
1105    2
2411    2
5114    1
3303    1
       ..
2504    1
2505    1
2506    1
2507    1
1210    1
Name: Sect_ID, Length: 415, dtype: int64

In [290]:
# Sector ID for Rwaniro shouldnt be 2411...it should be 2412
population[population['Sect_ID'] == 2411]

,Prov_ID,Province,Dist_ID,District,Sect_ID,Sector,Population of Sector,Unnamed: 7,Unnamed: 8
84,2,Amajyepfo,24,Huye,2411,Rusatira,"25,171.00",NaN,NaN
85,2,Amajyepfo,24,Huye,2411,Rwaniro,"21,595.00",NaN,NaN


In [291]:
govt_df[govt_df['Sect_ID'] == 2412]

,Prov_ID,Province,Dist_ID,District,Sect_ID,Sector
84,2,Amajyepfo,24,Huye,2412,Rwaniro


In [292]:
# Change Sector ID for Rwaniro to 2412

population.loc[85, 'Sect_ID'] = 2412

In [293]:
# make sure that the change is made in the population dataset now.
population[population['Sect_ID'] == 2412]

,Prov_ID,Province,Dist_ID,District,Sect_ID,Sector,Population of Sector,Unnamed: 7,Unnamed: 8
85,2,Amajyepfo,24,Huye,2412,Rwaniro,"21,595.00",NaN,NaN


In [294]:
# drop the duplicate values from the population dataset based on Sect_ID
population = population.drop_duplicates('Sect_ID')

# make sure there are no duplicates
population['Sect_ID'].value_counts()

5114    1
5113    1
4307    1
4308    1
4309    1
       ..
2504    1
2505    1
2506    1
2507    1
1210    1
Name: Sect_ID, Length: 416, dtype: int64

In [295]:
# merge the population data set with the bridge project to get the population by
# sector, included in the project data set

# drop the province, district, sector name and district, province ID from
# population dataset

population = population.drop(columns=['Prov_ID', 'Province', 'Dist_ID', 'District', 'Sector', 'Unnamed: 7', 'Unnamed: 8'])

# merge the two together, save new dataframe
current_df = pd.merge(current_df, population, on='Sect_ID')

In [296]:
current_df.head()

,Province,District,Sector,Bridge Site Name,Project Stage,Project Code,Bridge Type,GPS (Latitude),GPS (Longitude),Individuals Directly Served,Assessment Date,Original_Community_col,Community_Served,Prov_ID,Dist_ID,Sect_ID,Mean share of harvest sold,School_count,Population of Sector
0,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 1,Buzi,3,36,3605,20.1,6,"19,359.00"
1,Western Province,Rusizi,Giheke,Kamigisha,Rejected,1014106,Suspended,-2.42486,28.957259999999998,Unknown,2018-10-29,Community Served 1,Kabuga,3,36,3605,20.1,6,"19,359.00"
2,Western Province,Rusizi,Giheke,Gacyamo,Rejected,1014108,Suspension,-2.41855,28.969459999999998,Unknown,2018-10-29,Community Served 1,Gacyamo,3,36,3605,20.1,6,"19,359.00"
3,Western Province,Rusizi,Giheke,Nyakabanda,Rejected,1014105,Suspended,-2.42715,28.95369,Unknown,2018-10-29,Community Served 1,Kanoga,3,36,3605,20.1,6,"19,359.00"
4,Western Province,Rusizi,Giheke,Impala,Rejected,1014166,Unknown,-2.44246,28.9333,Unknown,2018-10-29,Community Served 1,Impala,3,36,3605,20.1,6,"19,359.00"


### Merge Electricity Access with bridge projects

In [297]:
electricity = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/Rwanda_electricity_access_districts.csv')

In [298]:
electricity.head()

,District,Access rate
0,Bugesera,0.61
1,Burera,0.41
2,Gakenke,0.32
3,Gasabo,0.57
4,Gatsibo,0.35


In [299]:
# update the ACcess rate to be in percent form by multiplying by 10
electricity['Access rate'] = electricity['Access rate'] * 100

electricity.head()

,District,Access rate
0,Bugesera,61.0
1,Burera,41.0
2,Gakenke,32.0
3,Gasabo,57.0
4,Gatsibo,35.0


In [300]:
# check to make sure we have the same districts in the electricity 
# as we do in the bridges projects dataset

sorted(electricity['District']) == sorted(set(current_df['District']))

True

In [301]:
# Because the districts are unique districts we can merge on the district name

current_df = pd.merge(current_df, electricity, on='District')



In [302]:
current_df.head()

,Province,District,Sector,Bridge Site Name,Project Stage,Project Code,Bridge Type,GPS (Latitude),GPS (Longitude),Individuals Directly Served,Assessment Date,Original_Community_col,Community_Served,Prov_ID,Dist_ID,Sect_ID,Mean share of harvest sold,School_count,Population of Sector,Access rate
0,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 1,Buzi,3,36,3605,20.1,6,"19,359.00",64.0
1,Western Province,Rusizi,Giheke,Kamigisha,Rejected,1014106,Suspended,-2.42486,28.957259999999998,Unknown,2018-10-29,Community Served 1,Kabuga,3,36,3605,20.1,6,"19,359.00",64.0
2,Western Province,Rusizi,Giheke,Gacyamo,Rejected,1014108,Suspension,-2.41855,28.969459999999998,Unknown,2018-10-29,Community Served 1,Gacyamo,3,36,3605,20.1,6,"19,359.00",64.0
3,Western Province,Rusizi,Giheke,Nyakabanda,Rejected,1014105,Suspended,-2.42715,28.95369,Unknown,2018-10-29,Community Served 1,Kanoga,3,36,3605,20.1,6,"19,359.00",64.0
4,Western Province,Rusizi,Giheke,Impala,Rejected,1014166,Unknown,-2.44246,28.9333,Unknown,2018-10-29,Community Served 1,Impala,3,36,3605,20.1,6,"19,359.00",64.0


### Merge Average Travel time to various Financial institutions

In [303]:
travel_time = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/avg_travel_time_financial_access.csv')

In [304]:
travel_time.head()

,District,Umurenge SACCO,MFIs,Bank Branch,ATM,Bank Agent,Mobile Money Agent
0,Nyarugenge,24:00,24:10,25:56,25:00,23:00,15:15
1,Gasabo,30:31,34:03,36:07,37:00,32:00,20:14
2,Kicukiro,30:13,28:00,30:00,29:55,28:21,17:00
3,Nyanza,55:00,58:00,62:00,61:00,59:03,40:00
4,Gisagara,51:23,60:23,55:00,54:19,55:00,44:00


In [305]:
current_df.shape

(4483, 20)

In [306]:
# merge the current_df with the travel times
current_df = pd.merge(current_df, travel_time, on='District')

current_df.head()

,Province,District,Sector,Bridge Site Name,Project Stage,Project Code,Bridge Type,GPS (Latitude),GPS (Longitude),Individuals Directly Served,Assessment Date,Original_Community_col,Community_Served,Prov_ID,Dist_ID,Sect_ID,Mean share of harvest sold,School_count,Population of Sector,Access rate,Umurenge SACCO,MFIs,Bank Branch,ATM,Bank Agent,Mobile Money Agent
0,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 1,Buzi,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21
1,Western Province,Rusizi,Giheke,Kamigisha,Rejected,1014106,Suspended,-2.42486,28.957259999999998,Unknown,2018-10-29,Community Served 1,Kabuga,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21
2,Western Province,Rusizi,Giheke,Gacyamo,Rejected,1014108,Suspension,-2.41855,28.969459999999998,Unknown,2018-10-29,Community Served 1,Gacyamo,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21
3,Western Province,Rusizi,Giheke,Nyakabanda,Rejected,1014105,Suspended,-2.42715,28.95369,Unknown,2018-10-29,Community Served 1,Kanoga,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21
4,Western Province,Rusizi,Giheke,Impala,Rejected,1014166,Unknown,-2.44246,28.9333,Unknown,2018-10-29,Community Served 1,Impala,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21


### Merge banking access dataset 

In [307]:
banking = pd.read_csv('https://raw.githubusercontent.com/Lambda-School-Labs/Labs25-Bridges_to_Prosperity-TeamC-ds/main/rwanda_data_files/datasets_22991_29451_rwa_exclusion.csv')

In [308]:
banking.rename(columns={'district': 'District'}, inplace=True)
banking.head()

,District,Banked,Other formal (non-bank),Informal only,Excluded
0,Nyarugenge,69,25,3,3
1,Kicukiro,58,36,3,3
2,Gasabo,52,36,5,7
3,Rwamagana,34,42,13,11
4,Ruhango,31,43,18,8


In [309]:
current_df = pd.merge(current_df, banking, on='District')

In [310]:
current_df.head()

,Province,District,Sector,Bridge Site Name,Project Stage,Project Code,Bridge Type,GPS (Latitude),GPS (Longitude),Individuals Directly Served,Assessment Date,Original_Community_col,Community_Served,Prov_ID,Dist_ID,Sect_ID,Mean share of harvest sold,School_count,Population of Sector,Access rate,Umurenge SACCO,MFIs,Bank Branch,ATM,Bank Agent,Mobile Money Agent,Banked,Other formal (non-bank),Informal only,Excluded
0,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 1,Buzi,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21
1,Western Province,Rusizi,Giheke,Kamigisha,Rejected,1014106,Suspended,-2.42486,28.957259999999998,Unknown,2018-10-29,Community Served 1,Kabuga,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21
2,Western Province,Rusizi,Giheke,Gacyamo,Rejected,1014108,Suspension,-2.41855,28.969459999999998,Unknown,2018-10-29,Community Served 1,Gacyamo,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21
3,Western Province,Rusizi,Giheke,Nyakabanda,Rejected,1014105,Suspended,-2.42715,28.95369,Unknown,2018-10-29,Community Served 1,Kanoga,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21
4,Western Province,Rusizi,Giheke,Impala,Rejected,1014166,Unknown,-2.44246,28.9333,Unknown,2018-10-29,Community Served 1,Impala,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21


### Get the number of communities that a specific bridge Serves

In [311]:
# get a dataframe that has the total number of communities served
communities_served = pd.DataFrame(current_df.groupby('Project Code', as_index=False).count()[['Project Code', 'Community_Served']])

communities_served.head()


,Project Code,Community_Served
0,1007327,2
1,1007328,1
2,1007329,2
3,1007330,1
4,1007331,4


In [312]:
communities_served = communities_served.rename(columns={'Community_Served': 'num_communities_served'})

In [313]:
current_df = pd.merge(current_df, communities_served, on='Project Code')

In [314]:
current_df.head()

,Province,District,Sector,Bridge Site Name,Project Stage,Project Code,Bridge Type,GPS (Latitude),GPS (Longitude),Individuals Directly Served,Assessment Date,Original_Community_col,Community_Served,Prov_ID,Dist_ID,Sect_ID,Mean share of harvest sold,School_count,Population of Sector,Access rate,Umurenge SACCO,MFIs,Bank Branch,ATM,Bank Agent,Mobile Money Agent,Banked,Other formal (non-bank),Informal only,Excluded,num_communities_served
0,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 1,Buzi,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,5
1,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 2,Kabuga,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,5
2,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 3,Kagarama,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,5
3,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 4,Gacyamo,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,5
4,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 5,Gasheke,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,5


### Drop the duplicates for project code:
this will give us the unique bridge

In [316]:
final_df = current_df.drop_duplicates(subset='Project Code')

In [317]:
final_df.head()

,Province,District,Sector,Bridge Site Name,Project Stage,Project Code,Bridge Type,GPS (Latitude),GPS (Longitude),Individuals Directly Served,Assessment Date,Original_Community_col,Community_Served,Prov_ID,Dist_ID,Sect_ID,Mean share of harvest sold,School_count,Population of Sector,Access rate,Umurenge SACCO,MFIs,Bank Branch,ATM,Bank Agent,Mobile Money Agent,Banked,Other formal (non-bank),Informal only,Excluded,num_communities_served
0,Western Province,Rusizi,Giheke,Buzi,Rejected,1014107,Suspended,-2.42056,28.9662,Unknown,2018-10-29,Community Served 1,Buzi,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,5
5,Western Province,Rusizi,Giheke,Kamigisha,Rejected,1014106,Suspended,-2.42486,28.957259999999998,Unknown,2018-10-29,Community Served 1,Kabuga,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,5
10,Western Province,Rusizi,Giheke,Gacyamo,Rejected,1014108,Suspension,-2.41855,28.969459999999998,Unknown,2018-10-29,Community Served 1,Gacyamo,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,5
15,Western Province,Rusizi,Giheke,Nyakabanda,Rejected,1014105,Suspended,-2.42715,28.95369,Unknown,2018-10-29,Community Served 1,Kanoga,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,5
20,Western Province,Rusizi,Giheke,Impala,Rejected,1014166,Unknown,-2.44246,28.9333,Unknown,2018-10-29,Community Served 1,Impala,3,36,3605,20.1,6,"19,359.00",64.0,38:21,49:27,53:26,52:00,50:17,32:21,27,38,14,21,1


In [320]:
final_df.to_csv('final_model_df.csv', index=False)

In [322]:
testing = pd.read_csv('/content/final_model_df.csv')

In [324]:
testing.dtypes

Province                        object
District                        object
Sector                          object
Bridge Site Name                object
Project Stage                   object
Project Code                     int64
Bridge Type                     object
 GPS (Latitude)                 object
GPS (Longitude)                 object
Individuals Directly Served     object
Assessment Date                 object
Original_Community_col          object
Community_Served                object
Prov_ID                          int64
Dist_ID                          int64
Sect_ID                          int64
Mean share of harvest sold     float64
School_count                     int64
 Population of Sector           object
Access rate                    float64
Umurenge SACCO                  object
MFIs                            object
Bank Branch                     object
ATM                             object
Bank Agent                      object
Mobile Money Agent       